In [36]:
## 먼저 데이터 불러오기
import json
import pandas as pd

# Step 1: Load the JSON data from the file
file_path = 'test.json'  # Replace this with the path to your JSON file
input_list = []
with open(file_path, 'r', encoding='utf-8') as file:
    json_data = json.load(file)

for item in json_data[:3]:
    input_list.append(item['text'])

In [37]:
## 실제로 데이터가 보이는지 확인
input_list

["엔트리움\r회사개요\r대표자 (생년) 정세영 (1973년)\r\r설립일\r\r2013. 2. 1\rsnuven@ntrium.com\r\rWebsite\r\rwww.ntrium.com\r\rE-mail\r\r운영사명\r\r캡스톤파트너스\r\r팁스 선정월\r\r사업화 단계 시리즈 B : 시장확대\r\r분\r\r소재지\r\r2013. 8월\r경기 화성시\r\r야\r\r전기ㆍ전자 - 디스플레이 - 디스플레이 부품 및 소재\r\r사 업 명\r\r비등방성 전도성 접착필름(ACF)용 도전성 입자 국산화\r\r사업내용\r\r? 나노융합소재 연구개발\r- 모바일기기, 반도체, 디스플레이 등 첨단 IT 제품용 핵심 나노소재 기술보유\r- 약물전달 시스템용 나노입자 개발 : 표적항암제 등 바이오 제약분야 기초기술 보유\r- 전기전도성 소재, 열전도성 소재, 전자파차폐 소재 개발 및 사업화\r수익창출 도전성 입자, EMI 차폐소재 등 납품을 통한 수익창출\r주요고객 SK하이닉스, UPAK(대만), SUNA(중국), 삼성전자, 앰코코리아,\r\r애플, ISC(한국), TSE(한국), H&S하이텍(한국), 회명산업 등\r\r비즈니스\r모\r델\r\r주요채널 국내외 고객사 직접 거래(B2B)\r핵심활동 도전성 입자 개발 및 이를 활용한 EMI 차폐소재 개발 및 납품 중\r\r본딩 필름, 자외선 차단 필름 등 협업 개발 및 납품 중\r해외진출 대만 고객사에 공급 중이며, 중국, 미국, 유럽, 일본 등 제품 평가 중\r\r사\r\r16년 약 12만 달러, 17년 약 15만 달러, 18년 24만 달러 수출\r\r업\r후속투자 122.3억원\r고용창출 (선정당시) 5명 -> ('22) 27명\r논문 및 특허 (특허) 국내외 출원 및 등록 38건\r매\r출 (‘22) 48억원\r기\r타 2014. 9. 소재부품기술개발사업 선정(투자자연계형-3년간 총 18억),\r\r개\r요\r\r주요성과\r\r2015. 1.\r2016.12.\r2017. 3.\r2017. 7.\r2017. 7.\r2017.

In [38]:
OPENAI_KEY = ""
from openai import OpenAI

client = OpenAI(
  api_key=OPENAI_KEY,
)

In [39]:
gpt_functions_2 = [
    {
        'name': 'extract_company_info',
        'description': 'Get the company information from the body of the input text. information should be in korean',
        'parameters': {
            'type': 'object',
            'properties': {
                'business_name': {
                    'type': 'string',
                    'description': 'Official registered business name.'
                },

                'email': {
                    'type': 'string',
                    'format': 'email',
                    'description': 'Contact email address for the company.'
                },
                'representative': {
                    'type': 'string',
                    'description': 'Name of the company representative.'
                },
                'website': {
                    'type': 'string',
                    'description': 'Official website of the company.'
                },

                'establishment_date': {
                    'type': 'string',  # Could be 'date' in specific formats
                    'description': 'The date when the company was established.'
                },

                'location': {
                    'type': 'string',
                    'description': 'Location of the company headquarters.'
                },
                'business_model': {
                    'type': 'string',
                    'description': 'The business model used by the company.'
                },
                'business_stage': {
                    'type': 'string',
                    'description': 'Current stage of business operations.'
                },
                'business_content': {
                    'type': 'string',
                    'description': 'Description of the business activities.'
                },
                'industry_field': {
                    'type': 'string',
                    'description': 'industry or field of the business.'
                },



            }
        }
    }
]

In [45]:

import time
import requests

## output is dict response
def get_response_with_retry(client, item, max_retries=3, delay=2):
    retries = 0
    while True:
        try:
            response = client.chat.completions.create(
                model='gpt-3.5-turbo',
                messages=[{'role': 'user', 'content': item}],
                functions=gpt_functions_2,
                function_call='auto'
            )
            
            # Assuming the API returns a JSON string that we want to parse
            dict_response = json.loads(response.choices[0].message.function_call.arguments)  # This line will raise JSONDecodeError if the response is not valid JSON
            return dict_response
            
        except Exception as e:
            # Catching all exceptions and retrying
            print(f"An exception occurred: {e}")
            retries += 1
            if retries >= max_retries:
                raise  # Re-raise the last exception after exceeding max retries
            print(f"Retrying in {delay} seconds...")
            time.sleep(delay)

# Usage of get_response_with_retry within the loop (example)
processed_list = []
for item in input_list:
    try:
        json_response = get_response_with_retry(client, item)
        processed_list.append(json_response)  # Append successful responses
    except Exception as e:
        print(f"Failed to process item after retries: {e}")




In [46]:
df = pd.DataFrame(processed_list)
df.to_excel("gpt3.5_240305_2.xlsx")